In [38]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [36]:
import pandas as pd
import sqlite3 as sql

conn = sql.connect('database.db')
path = 'term-project-sql/SourceData/'
course = '/courses.csv'
meeting = '/course_meetings.csv'

#list with semesters to loop path
semesters = ['Fall2014','Winter2015','Spring2015','Summer2015','Fall2015','Winter2016',
             'Spring2016','Summer2016','Fall2016','Winter2017','Spring2017','SpringBreak2017',
             'Summer2017','Fall2017','Winter2018','Spring2018','Summer2018','Fall2018',
             'Spring2019'
            ]


In [37]:
# Append both catalog csv files
df_catalog_2017_2018 = pd.read_csv(path + 'Catalogs/CourseCatalog2017_2018.csv')
df_catalog_2018_2019 = pd.read_csv(path + 'Catalogs/CourseCatalog2018_2019.csv')
df_catalogs = df_catalog_2017_2018.append(df_catalog_2018_2019)

#Start loop --> pull paths from cell above and swap "semester" with each value on the "semesters" list, appending the data to a single df each time around.
for semester in semesters:
    temp_course = pd.read_csv(path + semester + course)
    df_course = df_course.append(temp_course)
    
    temp_meeting = pd.read_csv(path + semester + meeting)
    df_meeting = df_meeting.append(temp_meeting)
    
#Send consolidated dataframes to csv --> next step is to merge these (and catalog as well) in order to mass dump into database
#After joining the dataframes, clean up the data prior to dumping into database

df_course.to_csv('term-project-sql/Data_Dump_course.csv')
df_meeting.to_csv('term-project-sql/Data_Dump_meeting.csv')

#reset df to blank after exporting Data_Dump.csv
df_course = pd.DataFrame()
df_meeting = pd.DataFrame()

print('Process Complete')



Process Complete
